**This Notebook prepares labels for the calssification of vertebrae based on the EfficientNet vertebrae classification model. This is then used as part of the inference of of the CT scans for the fracture prediction.**

## Load Package

In [1]:
#! pip install pylibjpeg pylibjpeg-libjpeg pydicom
#! pip install -U python-gdcm
#! pip install opencv-python-headless
#! pip install nibabel
#! pip install tensorflow
#! pip install --upgrade tensorflow_hub
#! pip install --upgrade tensorflow-gpu
#! pip install pydicom
#! pip install path.py

In [1]:
import sys
sys.path.append('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\kerasapplications')
sys.path.append('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\efficientnet-keras-source-code\\')
import keras_applications
import efficientnet.tfkeras as efficientnet

C:\Users\public.DESKTOP-P7554KD\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import cv2
import os
from os import listdir
import re
import gc
from time import time
from datetime import datetime

import gdcm
import pydicom as dicom
from pydicom import dcmread
import pylibjpeg
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom.data import get_testdata_files

from path import Path
import scipy.ndimage
from tqdm import tqdm
import nibabel as nib
from pprint import pprint
from time import time
import itertools
from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from glob import glob
import warnings
import dask.array as da

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, cohen_kappa_score, confusion_matrix
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import losses, callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as hub
from keras.preprocessing.image import ImageDataGenerator
from keras import layers

from PIL import Image as im
import traceback

import imageio


#from efficientnet_v2 import EfficientNetV2S
%matplotlib inline
sns.set(style='darkgrid', font_scale=1.6)

## Prepare Classification Labels

In [3]:
#Getting patient with mask
seg_paths = glob(f"C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\segmentations\\*")
training_patient=[]
for path in seg_paths:
    training_patient.append((path.rsplit("\\",1)[-1][:-4]))#Patient with mask present training_patient.append((path.rsplit("/",1)[-1])[:-4])

In [7]:
#Getting patient with mask
dicom_paths = glob("C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\train_images\\*")
all_patient=[]
for path in dicom_paths:
    all_patient.append((path.rsplit("\\",1)[-1]))#Patient with mask present

In [35]:
len(all_patient)

2019

In [27]:
len(infer_patient)

2019

In [39]:
all_patient[4]

'1.2.826.0.1.3680043.10032'

### Formating the label dataframe to be same format as fracture prediction model

In [8]:
def infer():
    
    patient_id = []
    image_id = []
    #for i in range(start,start+10):
    for i in range(len(all_patient[:300])):
        patient_ID=all_patient[i]
        print(patient_ID)
        dcm_paths = glob(f"C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\train_images\\{patient_ID}\\*")
        dcm_paths.sort(key=natural_keys)
    
        for paths in dcm_paths:
            patient_id.append(paths.split('\\')[-2])
            image_id.append(paths.split('\\')[-1][:-4])
        #infer_ds_label = [{'patient_id': paths.split('\\')[-2], 'image_id': paths.split('\\')[-1][:-4]} for paths in dcm_paths]
           
            
        
    return {'patient_id': patient_id, 'image_id': image_id}

In [9]:
infer_ds_label=infer()

1.2.826.0.1.3680043.10001
1.2.826.0.1.3680043.10005
1.2.826.0.1.3680043.10014
1.2.826.0.1.3680043.10016
1.2.826.0.1.3680043.10032
1.2.826.0.1.3680043.10041
1.2.826.0.1.3680043.10051
1.2.826.0.1.3680043.10058
1.2.826.0.1.3680043.10062
1.2.826.0.1.3680043.1010
1.2.826.0.1.3680043.10136
1.2.826.0.1.3680043.1016
1.2.826.0.1.3680043.10179
1.2.826.0.1.3680043.10204
1.2.826.0.1.3680043.10230
1.2.826.0.1.3680043.10261
1.2.826.0.1.3680043.10360
1.2.826.0.1.3680043.10400
1.2.826.0.1.3680043.10412
1.2.826.0.1.3680043.10431
1.2.826.0.1.3680043.10443
1.2.826.0.1.3680043.10449
1.2.826.0.1.3680043.10515
1.2.826.0.1.3680043.10520
1.2.826.0.1.3680043.10541
1.2.826.0.1.3680043.10579
1.2.826.0.1.3680043.10606
1.2.826.0.1.3680043.10608
1.2.826.0.1.3680043.10614
1.2.826.0.1.3680043.1062
1.2.826.0.1.3680043.10628
1.2.826.0.1.3680043.10633
1.2.826.0.1.3680043.10639
1.2.826.0.1.3680043.10673
1.2.826.0.1.3680043.10678
1.2.826.0.1.3680043.10697
1.2.826.0.1.3680043.1071
1.2.826.0.1.3680043.10725
1.2.826.0.1.3680

In [10]:
infer_label = pd.DataFrame.from_dict(infer_ds_label) 

In [11]:
infer_label

,patient_id,image_id
0,1.2.826.0.1.3680043.10001,1
1,1.2.826.0.1.3680043.10001,2
2,1.2.826.0.1.3680043.10001,3
3,1.2.826.0.1.3680043.10001,4
4,1.2.826.0.1.3680043.10001,5
...,...,...
106395,1.2.826.0.1.3680043.14669,263
106396,1.2.826.0.1.3680043.14669,264
106397,1.2.826.0.1.3680043.14669,265
106398,1.2.826.0.1.3680043.14669,266


In [12]:
infer_label.to_csv('infer_label.csv', index=False)